In [ ]:
!pip install geopandas
!pip install awscli

In [ ]:
import geopandas as gpd
import cuspatial
import cudf
from numba import cuda

In [ ]:
ook_url = "https://ookla-open-data.s3.amazonaws.com/shapefiles/performance/type=fixed/year=2020/quarter=1/2020-01-01_performance_fixed_tiles.zip"
county_url = "https://www2.census.gov/geo/tiger/TIGER2019/COUNTY/tl_2019_us_county.zip" 

In [ ]:
counties = gpd.read_file(county_url)

In [ ]:
counties.to_crs(epsg = 4326, inplace = True)
counries.to_file(')

In [ ]:
q1_2020 = gpd.read_file(ook_url)

In [ ]:
us_fixed = gpd.sjoin(q1_2020, counties, how = 'inner', op = 'intersects')

In [ ]:
us_fixed.to_file('Data/US_fixed.shp')

/opt/conda/envs/rapids/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  """Entry point for launching an IPython kernel.


In [ ]:
us_fixed.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
5,0231301222133033,166797,55407,18,38,10,"POLYGON ((-98.11340 29.71191, -98.10791 29.711...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
6466,0231301222120113,7850,1510,28,4,1,"POLYGON ((-98.22327 29.75961, -98.21777 29.759...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
9316,0231301222301320,116315,13823,22,83,20,"POLYGON ((-98.19580 29.65942, -98.19031 29.659...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
15655,0231301222311121,143093,15049,23,94,25,"POLYGON ((-98.10242 29.67851, -98.09692 29.678...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
15945,0231301222133230,118641,15201,22,36,7,"POLYGON ((-98.11890 29.69760, -98.11340 29.697...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440


In [ ]:
us_fixed.NAMELSAD.head()

5        Comal County
6466     Comal County
9316     Comal County
15655    Comal County
15945    Comal County
Name: NAMELSAD, dtype: object

In [ ]:
us_fixed.columns

Index(['quadkey', 'avg_d_kbps', 'avg_u_kbps', 'avg_lat_ms', 'tests', 'devices',
       'geometry', 'index_right', 'STATEFP', 'COUNTYFP', 'COUNTYNS', 'GEOID',
       'NAME', 'NAMELSAD', 'LSAD', 'CLASSFP', 'MTFCC', 'CSAFP', 'CBSAFP',
       'METDIVFP', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON'],
      dtype='object')

In [ ]:
counties.head()

,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry
0,31,039,00835841,31039,Cuming,Cuming County,06,H1,G4020,None,None,None,A,1477652222,10690952,+41.9158651,-096.7885168,"POLYGON ((-97.01952 42.00410, -97.01952 42.004..."
1,53,069,01513275,53069,Wahkiakum,Wahkiakum County,06,H1,G4020,None,None,None,A,680962890,61582307,+46.2946377,-123.4244583,"POLYGON ((-123.43640 46.23819, -123.44759 46.2..."
2,35,011,00933054,35011,De Baca,De Baca County,06,H1,G4020,None,None,None,A,6016819475,29089486,+34.3592729,-104.3686961,"POLYGON ((-104.56739 33.99756, -104.56772 33.9..."
3,31,109,00835876,31109,Lancaster,Lancaster County,06,H1,G4020,339,30700,None,A,2169270569,22849484,+40.7835474,-096.6886584,"POLYGON ((-96.91075 40.78494, -96.91075 40.790..."
4,31,129,00835886,31129,Nuckolls,Nuckolls County,06,H1,G4020,None,None,None,A,1489645188,1718484,+40.1764918,-098.0468422,"POLYGON ((-98.27367 40.08940, -98.27367 40.089..."


In [ ]:
county_grouped = us_fixed.groupby('NAMELSAD')

In [ ]:
county_grouped.head()

,quadkey,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,index_right,STATEFP,COUNTYFP,...,CLASSFP,MTFCC,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON
5,0231301222133033,166797,55407,18,38,10,"POLYGON ((-98.11340 29.71191, -98.10791 29.711...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
6466,0231301222120113,7850,1510,28,4,1,"POLYGON ((-98.22327 29.75961, -98.21777 29.759...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
9316,0231301222301320,116315,13823,22,83,20,"POLYGON ((-98.19580 29.65942, -98.19031 29.659...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
15655,0231301222311121,143093,15049,23,94,25,"POLYGON ((-98.10242 29.67851, -98.09692 29.678...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
15945,0231301222133230,118641,15201,22,36,7,"POLYGON ((-98.11890 29.69760, -98.11340 29.697...",703,48,091,...,H1,G4020,484,41700,None,A,1449162144,39775861,+29.8054913,-098.2605440
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6048530,0213231110113113,1709,3074,219,1,1,"POLYGON ((-101.60706 45.05412, -101.60156 45.0...",504,46,137,...,H1,G4020,None,None,None,A,5079588972,24212302,+44.9897638,-101.6608274
6372959,0213231110113131,1010,2172,662,1,1,"POLYGON ((-101.60706 45.05024, -101.60156 45.0...",504,46,137,...,H1,G4020,None,None,None,A,5079588972,24212302,+44.9897638,-101.6608274
4700999,0213321203213321,26157,4121,38,4,1,"POLYGON ((-97.92664 43.65198, -97.92114 43.651...",1074,46,061,...,H1,G4020,None,33580,None,A,1125574352,1991676,+43.6806116,-097.7968454
5405474,0213321203211202,3367,685,76,7,1,"POLYGON ((-97.95410 43.68774, -97.94861 43.687...",1074,46,061,...,H1,G4020,None,33580,None,A,1125574352,1991676,+43.6806116,-097.7968454


In [ ]:
county_grouped.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 3219 entries, 00023901 to 02516404
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   avg_d_kbps   3219 non-null   float64
 1   avg_u_kbps   3219 non-null   float64
 2   avg_lat_ms   3219 non-null   float64
 3   tests        3219 non-null   float64
 4   devices      3219 non-null   float64
 5   index_right  3219 non-null   float64
 6   ALAND        3219 non-null   float64
 7   AWATER       3219 non-null   float64
dtypes: float64(8)
memory usage: 226.3+ KB
